<a href="https://colab.research.google.com/github/Diegostvz/visualizacion-datos/blob/main/ejercicio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
pip install plotly_express

In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [27]:
url = 'https://raw.githubusercontent.com/Diegostvz/visualizacion-datos/main/ventas_composicion.csv'
df = pd.read_csv(url)
print(df)


          Fecha Categoría  Ventas  Porcentaje
0    2020-01-01         A    8270       36.41
1    2020-01-01         B    1860        8.19
2    2020-01-01         C    6390       28.14
3    2020-01-01         D    6191       27.26
4    2020-02-01         A    6734       32.23
..          ...       ...     ...         ...
235  2024-11-01         D    1098        5.31
236  2024-12-01         A    3200       17.23
237  2024-12-01         B    3961       21.33
238  2024-12-01         C    5973       32.17
239  2024-12-01         D    5433       29.26

[240 rows x 4 columns]


## ¿Cómo ha evolucionado la cuota de mercado de cada categoría a lo largo del tiempo?

- mensaje: mostrar la evolución de la variable porcentaje de cada categoria con respecto al tiempo(la fecha en este caso)

- los datos a explicar se componen en 4 variables son: fecha(dia en que se produjo la venta), categoría(a que clase pertenece las ventas), ventas(numero de ventas del dia), porcentaje(cuota con repecto al total)

- persona: el gráfico va dirigido al profesor de la asignatura visualización de datos

- limitantes: tiempo para realizar el ejercicio y formato del mismo ya que hay muchas opciones


Se elige el gráfico filled_area_plot ya que permite representar la evolución temporal de una variable con respecto a un total(siendo la suma de dicha variable = 100%)

In [28]:
fig = px.area(df, x="Fecha", y="Porcentaje", color="Categoría", title="Evolución cuota de mercado con el tiempo")
fig.show()

## ¿Qué categoría tiene el comportamiento más estacional a lo largo del año?

- mensaje: mostrar el elemento más estacional, es decir el que menos variación tiene entre el maximo valor y el minimo valor en intervalos de tiempo igualmente espaciados

- los datos a explicar son los mismos que en el primer apartado

- persona: el gráfico va dirigido al profesor de la asignatura visualización de datos

- limitantes: tiempo para realizar el ejercicio y formato del mismo ya que hay muchas opciones

Se usa el mismo tipo de gráfico que en el apartado a, pero en este caso vamos a pintar 4 gráficos, uno por categoría, en particular, en el eje 'y' estará el numero de ventas y en el eje 'x' los meses, siendo cada color del filled_area_plot un año de la categoría correspondiente.

In [29]:
df["Fecha"] = pd.to_datetime(df["Fecha"])
df["Año"] = df["Fecha"].dt.year
df["Mes"] = df["Fecha"].dt.month

ventas_mensuales = df.groupby(["Categoría", "Año", "Mes"])["Ventas"].sum().reset_index()

for cat in ventas_mensuales["Categoría"].unique():
    df_cat = ventas_mensuales[ventas_mensuales["Categoría"] == cat]
    fig = px.area(df_cat,
        x="Mes",
        y="Ventas",
        color="Año",
        line_group="Año",
        groupnorm=None,
        title=f"Ventas mensuales categoría {cat}",
        labels={"Mes": "Mes", "Ventas": "Ventas"},
        template="plotly_white"
    )
    fig.show()


## ¿En qué momentos una categoría supera a otra en cuota de ventas?

- mensaje: mostrar la evolucion de la cuota de ventas haciendo enfasis en los cruces de las mismas

- los datos a explicar son los mismos que en el primer apartado

- persona: el gráfico va dirigido al profesor de la asignatura visualización de datos

- limitantes: tiempo para realizar el ejercicio y formato del mismo ya que hay muchas opciones

Se divide por año, se calculan las cuota de ventas mensual de cada categoria, para reducir el número de sobrepasos entre categorías y se usa para una graficación simple un gráfico de barras apiladas, ya que considero que usar otro tipo como un bump chart, al haber tantos cruces, es difícil de entender, incumpliendo el criterio de simplicidad.

In [30]:
ventas_mes = df.groupby(["Año", "Mes", "Categoría"])["Ventas"].sum().reset_index()
total_mes = ventas_mes.groupby(["Año", "Mes"])["Ventas"].sum().reset_index().rename(columns={"Ventas": "TotalVentas"})
ventas_mes = ventas_mes.merge(total_mes, on=["Año", "Mes"])
ventas_mes["Cuota"] = ventas_mes["Ventas"] / ventas_mes["TotalVentas"] * 100

#grafico para cada año(sino queda muy poco entendible por haber muchas barras)
for year in sorted(ventas_mes["Año"].unique()):
    df_year = ventas_mes[ventas_mes["Año"] == year]
    fig = px.bar(
        df_year,
        x="Mes",
        y="Cuota",
        color="Categoría",
        barmode="group",
        title=f"Cuota de ventas por categoría en el año {year}",
        labels={"Cuota": "Cuota de ventas (%)", "Mes": "Mes"},
    )
    fig.update_layout(xaxis_tickformat="%b") #linea sugerida por gemini en colab, para una mejor visualizacion de las etiquetas
    fig.show()


## ¿Cómo cambia la participación de las categorías si agrupamos los datos por año en lugar de por mes?

- mensaje: mostrar la evolucion de la cuota de ventas en este caso agrupando en años

- los datos a explicar son los mismos que en el primer apartado

- persona: el gráfico va dirigido al profesor de la asignatura visualización de datos

- limitantes: tiempo para realizar el ejercicio y formato del mismo ya que hay muchas opciones

Al agrupar por años, se reduce mucho el número de elementos en el gráfico y por ello el número de sobrepasos en categorías ya que solo existen ahora 4 periodos donde comparar.

Se usa un barchart siguiendo la lógica del apartado anterior, y además se añade también un bump chart ya que en este caso el número de elementos reducido permite un mejor entendimiento usando un gráfico que represente la evolución temporal y jerarquía entre elementos.

In [31]:
# claculamos las ventas anuales y la cueta
ventas_anual = (
    df.groupby(["Año", "Categoría"])["Ventas"]
    .sum()
    .reset_index()
)
total_anual = ventas_anual.groupby("Año")["Ventas"].sum().reset_index().rename(columns={"Ventas": "TotalAnual"})
ventas_anual = ventas_anual.merge(total_anual, on="Año")
ventas_anual["CuotaAnual"] = ventas_anual["Ventas"] / ventas_anual["TotalAnual"] * 100

In [32]:
#grafico bar chart
fig = px.bar(
    ventas_anual,
    x="Año",
    y="CuotaAnual",
    color="Categoría",
    barmode="group",
    title="Participación anual de las categorías en cuota de ventas",
    labels={"CuotaAnual": "Cuota de ventas anual (%)", "Año": "Año"}
)
fig.show()


In [33]:
# Calcular posición ocupada en cada año según cuota (ranking)
ventas_anual["Rango"] = ventas_anual.groupby("Año")["CuotaAnual"].rank(method="dense", ascending=False)

# Crear bump chart usando el ranking
fig = px.line(
    ventas_anual,
    x="Año",
    y="Rango",
    color="Categoría",
    line_shape="spline",
    markers=True,
    title="Bump chart de participación anual por categorías"
)

fig.update_yaxes(autorange="reversed", dtick=1, title="Ranking")
fig.update_layout(hovermode="x unified")
fig.show()